In [1]:
import os.path
import urllib.request
import re
import pandas as pd
import json
import warnings

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

In [3]:
def import_dict(metadata):
    with open(metadata) as myfile:
    	indep_contents = myfile.read()
    return json.loads(indep_contents)

def import_dataset(dataset):
    df0 = pd.read_sas(dataset)
    print(f"Number of original data: {len(df0)}")
    df0 = df0[df0['COV']!=0]
    print(f"An infant born after calendar year (COV = 0) is excluded")
    print(f"Number of training data: {len(df0)}")
    return df0

def code_df(df0, indep_dict, dep_attrs):
    df1 = pd.DataFrame(index = df0.index)
    for v in indep_dict.keys():
        if indep_dict[v]['type'] == 'Categorical':
            df1[v] = df0[v].astype('int8').astype('category')
        elif max(df0[v]) <= 65504 and min(df0[v]) >= -65504:
            df1[v] = df0[v].astype('float16')
        else:
            df1[v] = df0[v].astype('float32')
    df1[dep_attrs] = df0[dep_attrs].astype('int8')
    df1['class'] = 0
    df1['code'] = ""
    for v in dep_attrs:
    	df1[v] = df1[v].replace([2.0, 1.0], [False, True])
    	df1['class'] = 2*df1['class'] + df1[v]
    	df1['code'] = df1['code'] + df1[v].replace([True, False], ['Y', 'N'])
    df1[dep_attrs] = df1[dep_attrs].astype('category')
    df1['class'] = df1['class'].astype('int8').astype('category')
    df1['code'] = df1['code'].astype('category')
    return df1

def recode_df(df1):
    df2 = df1.copy()
    df2['code_new'] =  df2['code'].apply(
        lambda v: 'NY_' if re.match('(NY)', v) 
        else 'Y1Y' if re.match('^Y(?:\w*Y)', v)  
        else v
    )
    df2['code_new'] = df2['code_new'].astype('category')
    df2['class_new'] =  df2[['class', 'code_new']].apply(
        lambda v: 2 if v['code_new'] == 'NY_'
        else 3 if v['code_new'] == 'YNN'
        else 4 if v['code_new'] == 'Y1Y'
        else v['class'], 
        axis=1
    )
    df2['class_new'] = df2['class_new'].astype('int8').astype('category')
    return df2

def type_df(df, option='short'):
    if option.lower() == 'full':
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(df.dtypes)
    else:
        print(df.dtypes)  

In [4]:
if not os.path.isfile('pppub20.sas7bdat'):
    urllib.request.urlretrieve('https://filedn.eu/lrP6X4gzwLByok9WpRTqaG8/pppub20.sas7bdat', 'pppub20.sas7bdat')

In [5]:
indep_dict = import_dict(metadata='meta-indep.json')
dep_attrs = ['GRP', 'DIR', 'PUB']
df0 = import_dataset(dataset='pppub20.sas7bdat')

Number of original data: 157959
An infant born after calendar year (COV = 0) is excluded
Number of training data: 157681


In [6]:
num_cat = 0
num_cont = 0
for key in indep_dict:
    if indep_dict[key]['type'] == 'Categorical':
        num_cat += 1
    else:
        num_cont += 1
print(f"There are {num_cat + num_cont} independent variables of interest: {num_cat} categorical and {num_cont} continuous")

There are 184 independent variables of interest: 118 categorical and 66 continuous


In [7]:
df1 = code_df(df0, indep_dict, dep_attrs)
df = recode_df(df1)

In [8]:
type_df(df, option='full')

A_AGE            float16
A_EXPRRP        category
A_FAMTYP        category
A_HGA           category
A_MARITL        category
A_PFREL         category
A_SEX           category
P_STAT          category
PEAFEVER        category
PEDISDRS        category
PEDISEAR        category
PEDISEYE        category
PEDISOUT        category
PEDISPHY        category
PEDISREM        category
PRDISFLG        category
PRCITSHP        category
PRDTRACE        category
A_MJIND         category
A_MJOCC         category
PEIO1COW        category
PRDISC          category
PRUNTYPE        category
A_GRSWK          float16
A_HRLYWK        category
A_HRSPAY         float16
PRERELG         category
A_CIVLF         category
A_CLSWKR        category
A_EXPLF         category
A_LFSR          category
A_UNCOV         category
A_UNMEM         category
A_UNTYPE        category
A_USLHRS         float16
A_WKSCH         category
A_WKSLK          float16
A_WKSTAT        category
PEHRUSLT         float16
PEMLR           category


In [9]:
df[['GRP','DIR','PUB','class','code','code_new','class_new']].drop_duplicates().sort_values('class').reset_index(drop=True)

,GRP,DIR,PUB,class,code,code_new,class_new
0,False,False,False,0,NNN,NNN,0
1,False,False,True,1,NNY,NNY,1
2,False,True,False,2,NYN,NY_,2
3,False,True,True,3,NYY,NY_,2
4,True,False,False,4,YNN,YNN,3
5,True,False,True,5,YNY,Y1Y,4
6,True,True,False,6,YYN,Y1Y,4
7,True,True,True,7,YYY,Y1Y,4


In [10]:
print(f"Code: Employment-based plan (GRP) | Direct-purchase plan (DIR) | Public health insurance (PUB)")
print("\n")
print(df1.groupby('code').size())
print("\n")
print(df.groupby('code_new').size())
print("\n-----------------------------------------")

Code: Employment-based plan (GRP) | Direct-purchase plan (DIR) | Public health insurance (PUB)


code
NNN    15035
NNY    38745
NYN     8084
NYY     6157
YNN    80165
YNY     8206
YYN     1104
YYY      185
dtype: int64


code_new
NNN    15035
NNY    38745
NY_    14241
Y1Y     9495
YNN    80165
dtype: int64

-----------------------------------------


In [11]:
for v in indep_dict.keys():
    if indep_dict[v]['type'] == "Categorical":
        dat_tab = pd.crosstab(index=df[v].map(lambda x: indep_dict[v]['values'][str(x)]), columns=df['code_new'])
    else:
        dat = df[[v, 'code_new']].copy()
        bins = 10
        dat['bins'] = pd.cut(dat[v], bins)
        dat_tab = pd.crosstab(index=dat['bins'],columns=dat['code_new'])
        del dat
    print(v)
    print(f"Label: {indep_dict[v]['label']}")
    print(f"Universe: {indep_dict[v]['universe']}")
    print(f"Type: {indep_dict[v]['type']}")
    print(f"Topic: {indep_dict[v]['topic']}")
    print(f"Subtopic: {indep_dict[v]['subtopic']}")
    print(f"Code: Employment-based plan (GRP) | Direct-purchase plan (DIR) | Public health insurance (PUB)")
    print(dat_tab)
    print("\n-----------------------------------------")
    del dat_tab

A_AGE
Label: Age
Universe: All Persons
Type: Continuous
Topic: Demographics
Subtopic: Individual characteristics
Code: Employment-based plan (GRP) | Direct-purchase plan (DIR) | Public health insurance (PUB)
code_new        NNN   NNY   NY_   Y1Y    YNN
bins                                        
(-0.085, 8.5]  1407  5834   789   628   9795
(8.5, 17.0]    1557  6237  1079   770  11822
(17.0, 25.5]   2238  2475  1043   414   8017
(25.5, 34.0]   2635  2749  1082   594  10611
(34.0, 42.5]   2271  2146   976   613  11509
(42.5, 51.0]   2109  2171  1157   518  12081
(51.0, 59.5]   1606  2403  1223   471   9864
(59.5, 68.0]   1028  4854  2313  2090   6097
(68.0, 76.5]    105  5404  2602  2044    254
(76.5, 85.0]     79  4472  1977  1353    115

-----------------------------------------
A_EXPRRP
Label: Expanded relationship code
Universe: All Persons
Type: Categorical
Topic: Demographics
Subtopic: Individual characteristics
Code: Employment-based plan (GRP) | Direct-purchase plan (DIR) | Publ